In [ ]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

In [ ]:
for index, row in df.iterrows():
    parameters = {
        # 'candidate': [2, 3, 4, 5],
        # 'dropout': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
        'samples': [50, 100, 150, 200, 250, 300, 400, 600, 1000, 1500, 2000, 3000, 4000],
        # 'dimensions': [50, 100, 200],
        # 'iterations': [40, 80, 120, 160, 200],
    }
    
    print('doc_id: %3s actual author: %4s author_id: %s' % (str(doc_id), str(row.author_id), str(authorList)))
    
    level = 'word'
    
    for idxp, paralist in parameters.iteritems():
        
        if idxp == 'samples':
            for idxl, val in enumerate(paralist):
                
                doc_id = row.doc_id

                author_id = row.author_id

                import ast
                authorList = ast.literal_eval(row.authorList)

                val = len(authorList)

                test = "samples" # change before run

                level = "word"

                iterations = 30

                dropout = 0.4

                samples = 300

                loc = authorList.index(author_id)

                printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(val)))
                printstate += (("dimensions = %s, samples = %s, ") % (str(200), str(samples)))
                printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(iterations), str(dropout), str(test)))

                print("Current test: %s" % (str(printstate)))

                import UpdateDB as db

                case = db.checkOldCNN(doc_id = doc_id, candidate = val, dimensions = 200, samples = 300, 
                                      iterations = iterations, dropout = dropout, test = test)

                if case == False:

                    print("Running: %12s" % (str(printstate)))

                    import StyloNeural as Stylo
                    (labels_index, predYList, predY, history, samples) = Stylo.getResults(
                        doc_id = doc_id, authorList = authorList[:], 
                        level = level, glove = '../../glove/', dimensions = 200, 
                        samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

                    for key, auth in labels_index.iteritems():
                        if auth == author_id:
                            loc = key

                    print("Loc = %s" % (str(loc)))

                    ans = 0.0

                    if (predY.tolist().index(max(predY)) == loc):
                        ans = 1.0

                    import UpdateDB as db
                    case = db.updateresultOldCNN(doc_id = doc_id, candidate = val, dimensions = 200, samples = samples, 
                                                 iterations = iterations, dropout = dropout, accuracy = predY[loc], 
                                                 test = test)

                    if case == False:
                        print("Completed: %12s" % (str(printstate)))

                    else:
                        print("Skipped: %12s" % (str(printstate)))

                    from IPython.display import clear_output
                    clear_output()

                else:
                    print("Skipped: %12s" % (str(printstate)))
            

In [ ]:
%tb